In [1]:
# ! pip install dash   
# ! pip install dash-html-components                                         
# ! pip install dash-core-components                                     
# ! pip install plotly
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
from dash.dependencies import Input, Output
import numpy as np
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import json
import folium
import requests
import mimetypes
import http.client
import numpy as np
import matplotlib.pyplot as plt
from folium.plugins import HeatMap
from pandas.io.json import json_normalize
import seaborn as sns

#COVID
conn= http.client.HTTPSConnection("api.covid19api.com")
payload=''
headers={}
conn.request("GET","/summary",payload,headers)
res=conn.getresponse()
data=res.read().decode('UTF-8')
pd.set_option("display.max_rows", None, "display.max_columns", None)
covid1=json.loads(data)
pd.json_normalize(covid1['Countries'],sep=",")
df=pd.DataFrame(covid1['Countries'])
covid2=df.drop(columns=['ID','CountryCode','Slug','Date','Premium'],axis=1)
covid2=covid2.sort_values(by=['TotalConfirmed'],ascending=False)
covid2['Country'] = covid2['Country'].replace('Russian Federation', 'Russia')
covid2['Country'] = covid2['Country'].replace('Iran, Islamic Republic of', 'Iran')
covid2['Country'] = covid2['Country'].replace('Venezuela (Bolivarian Republic)', 'Venezuela')
covid3=covid2

# SARS
sars_03 = pd.read_csv(r"C:\Users\vansh\Desktop\DV-CSE3020\DV_Project\DV_Project\sars_2003_complete_dataset_clean.csv",parse_dates=['Date'])
sars_03 = sars_03[['Date', 'Country', 'Cumulative number of case(s)','Number of deaths', 'Number recovered']]
sars_03.columns = ['Date', 'Country', 'Cases', 'Deaths', 'Recovered']
sars_03 = sars_03.groupby(['Date', 'Country'])
sars_03 = sars_03.sum().reset_index()
sars_03['Cases'] = sars_03['Cases'].fillna(0)
sars_03['Deaths'] = sars_03['Deaths'].fillna(0)
sars_03['Recovered'] = sars_03['Recovered'].fillna(0)
sars_latest = sars_03[sars_03['Date'] == max(sars_03['Date'])].reset_index()
sars_latest=sars_latest.sort_values(by=['Cases'],ascending=False)

# H1N1
h1n1_09 = pd.read_csv(r"C:\Users\vansh\Desktop\DV-CSE3020\DV_Project\DV_Project\Pandemic_H1N1_2009.csv",parse_dates=['Update Time'])
h1n1_09 = h1n1_09[['Update Time', 'Country', 'Cases','Deaths']]
h1n1_09 = h1n1_09.groupby(['Update Time', 'Country'])
h1n1_09 = h1n1_09.sum().reset_index()
h1n1_09['Cases'] = h1n1_09['Cases'].fillna(0)
h1n1_09['Deaths'] = h1n1_09['Deaths'].fillna(0)
h1n1_09['Deaths'] = h1n1_09['Deaths'].astype('int')
h1n1_latest=h1n1_09[h1n1_09['Update Time'] == max(h1n1_09['Update Time'])].reset_index()
h1n1_latest=h1n1_latest.sort_values(by=['Cases'],ascending=False)
h1n1_latest=h1n1_latest.drop([43])

# EBOLA
ebola_14 = pd.read_csv(r"C:\Users\vansh\Desktop\DV-CSE3020\DV_Project\DV_Project\ebola_2014_2016_clean.csv",parse_dates=['Date'])
ebola_14 = ebola_14[['Date', 'Country',
 'Cumulative no. of confirmed, probable and suspected cases','Cumulative no. of confirmed, probable and suspected deaths']]
ebola_14.columns = ['Date', 'Country', 'Cases', 'Deaths']
ebola_14 = ebola_14.groupby(['Date', 'Country'])
ebola_14 = ebola_14.sum().reset_index()
ebola_14['Cases'] = ebola_14['Cases'].fillna(0)
ebola_14['Deaths'] = ebola_14['Deaths'].fillna(0)
ebola_14['Cases'] = ebola_14['Cases'].astype('int')
ebola_14['Deaths'] = ebola_14['Deaths'].astype('int')
ebola_latest = ebola_14[ebola_14['Date'] == max(ebola_14['Date'])].reset_index()
ebola_latest=ebola_latest.sort_values(by=['Cases'],ascending=False)

#MERS
mers_latest = pd.read_csv(r"C:\Users\vansh\Desktop\DV-CSE3020\DV_Project\DV_Project\country_count_latest.csv")
mers_latest=mers_latest.sort_values(by=['Confirmed'],ascending=False)

coordinates=pd.read_csv(r"C:\Users\vansh\Desktop\DV-CSE3020\DV_Project\DV_Project\world_country_and_usa_states_latitude_and_longitude_values.csv")
coordinates = coordinates.rename(columns={'country': 'Country'})
coordinates= coordinates.drop(columns=['country_code', 'usa_state_code','usa_state_latitude','usa_state_longitude','usa_state'])

def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                        radius=5,
                        weight=2,
                        popup=[point.Country,point.TotalConfirmed,point.TotalRecovered],
                        fill_color='#000000').add_to(m)
##########################################################################################################################
############################                               TREEMAP                        ################################
##########################################################################################################################

fig = px.treemap(covid3, names='Country', parents=[""]*len(covid3), values='TotalConfirmed')
fig.add_trace(go.Treemap(
    labels = list(covid3.Country),
    parents = [""]*len(covid3),
    values =  list(covid3.TotalConfirmed),
    textinfo = "label+value+percent parent",
    ),row = 1, col = 1)
covid_treemap=fig

h_df=h1n1_latest
fig = px.treemap(h_df, names='Country', parents=[""]*len(h_df), values='Cases')
fig.add_trace(go.Treemap(
    labels = list(h_df.Country),
    parents = [""]*len(h_df),
    values =  list(h_df.Cases),
    textinfo = "label+value+percent parent",
    ),row = 1, col = 1)
h1n1_treemap=fig

s_df=sars_latest
fig = px.treemap(s_df, names='Country', parents=[""]*len(s_df), values='Cases')
fig.add_trace(go.Treemap(
    labels = list(s_df.Country),
    parents = [""]*len(s_df),
    values =  list(s_df.Cases),
    textinfo = "label+value+percent parent",
    ),row = 1, col = 1)
sars_treemap=fig

m_df=mers_latest
fig = px.treemap(m_df, names='Country', parents=[""]*len(m_df), values='Confirmed')
fig.add_trace(go.Treemap(
    labels = list(m_df.Country),
    parents = [""]*len(m_df),
    values =  list(m_df.Confirmed),
    textinfo = "label+value+percent parent",
    ),row = 1, col = 1)
mers_treemap=fig

e_df=ebola_latest
fig = px.treemap(e_df, names='Country', parents=[""]*len(e_df), values='Cases')
fig.add_trace(go.Treemap(
    labels = list(e_df.Country),
    parents = [""]*len(e_df),
    values =  list(e_df.Cases),
    textinfo = "label+value+percent parent",
    ),row = 1, col = 1)
ebola_treemap=fig
#############################################################################################################################

#############################################################################################################################
######################################            GEOSPATIAL MAP          ###################################################
#############################################################################################################################

# H1N1 
m6=folium.Map(tiles="Stamen Terrain",min_zoom=1.5)
url='https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes=f'{url}/world-countries.json'
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='H1N1',
    data=h1n1_latest,
    columns=['Country','Cases'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='white',
    legend_name='Total Confirmed H1N1 Cases',
).add_to(m6)
h1n1_final=pd.merge(h1n1_latest,coordinates,on='Country')

def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                        radius=5,
                        weight=2,
                        popup=[point.Country,point.Cases,point.Deaths],
                        fill_color='#000000').add_to(m6)
    
h1n1_final.apply(plotDot,axis=1)
m6.fit_bounds(m6.get_bounds())
h1n1_geospatialmap=m6


# SARS
m4=folium.Map(tiles="Stamen Terrain",min_zoom=1.5)
url='https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes=f'{url}/world-countries.json'
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='SARS',
    data=sars_latest,
    columns=['Country','Cases'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='white',
    legend_name='Total Confirmed SARS Cases',
).add_to(m4)
sars_final=pd.merge(sars_latest,coordinates,on='Country')

def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                        radius=5,
                        weight=2,
                        popup=[point.Country,point.Cases,point.Deaths],
                        fill_color='#000000').add_to(m4)

sars_final.apply(plotDot,axis=1)
m4.fit_bounds(m4.get_bounds())
sars_geospatialmap=m4

# MERS
m5=folium.Map(tiles="Stamen Terrain",min_zoom=1.5)
url='https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes=f'{url}/world-countries.json'
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='MERS',
    data=mers_latest,
    columns=['Country','Confirmed'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='white',
    legend_name='Total Confirmed MERS Cases',
).add_to(m5)
mers_final=pd.merge(mers_latest,coordinates,on='Country')

def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                        radius=5,
                        weight=2,
                        popup=[point.Country,point.Confirmed],
                        fill_color='#000000').add_to(m5)
    
mers_final.apply(plotDot,axis=1)
m5.fit_bounds(m5.get_bounds())
mers_geospatialmap=m5

#EBOLA
m3=folium.Map(tiles="Stamen Terrain",min_zoom=1.5)
url='https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes=f'{url}/world-countries.json'
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='EBOLA',
    data=ebola_latest,
    columns=['Country','Cases'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='white',
    legend_name='Total Confirmed EBOLA Cases',
).add_to(m3)
ebola_final=pd.merge(ebola_latest,coordinates,on='Country')

def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                        radius=5,
                        weight=2,
                        popup=[point.Country,point.Cases,point.Deaths],
                        fill_color='#000000').add_to(m3)
    
ebola_final.apply(plotDot,axis=1)
m3.fit_bounds(m3.get_bounds())
ebola_geospatialmap=m3

#COVID
m=folium.Map(tiles="Stamen Terrain",min_zoom=1.5)
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='Covid-19',
    data=covid3,
    columns=['Country','TotalConfirmed'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='white',
    legend_name='Total Confirmed Covid Cases',
).add_to(m)
covid_final=pd.merge(covid3,coordinates,on='Country')
def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                        radius=5,
                        weight=2,
                        popup=[point.Country,point.TotalConfirmed,point.TotalRecovered],
                        fill_color='#000000').add_to(m)
    
covid_final.apply(plotDot,axis=1)
m.fit_bounds(m.get_bounds())
covid_geospatialmap = m
#############################################################################################################################

#############################################################################################################################
######################################            STATISTICAL FIG          ###################################################
#############################################################################################################################

df = pd.read_csv(r'C:\Users\vansh\Desktop\DV-CSE3020\DV_Project\DV_Project\PANDEMIC.csv')
df.sort_values('CONFIRMED', ascending =False)

color = ['aqua', 'lightcoral', 'yellowgreen', 'mediumpurple', 'mediumturquoise']
rcolor = ['mediumturquoise', 'mediumpurple', 'yellowgreen', 'lightcoral', 'aqua']
df2 = df.sort_values('DEATHS')
fig = px.bar(df2, x='EPIDEMIC', y='CONFIRMED',color = "EPIDEMIC")
confirmed_bar=fig

df2 = df.sort_values('CONFIRMED')
fig = px.bar(df2, x='EPIDEMIC', y='CONFIRMED',color = "EPIDEMIC", log_y=True)
confirmed_log_bar=fig

df2 = df.sort_values('CONFIRMED')
fig = px.bar(df2, x='EPIDEMIC', y='CONFIRMED',color = "EPIDEMIC")
death_bar=fig

df2 = df.sort_values('CONFIRMED')
fig = px.bar(df2, x='EPIDEMIC', y='CONFIRMED',color = "EPIDEMIC", log_y=True)
death_log_bar=fig

df2 = df.sort_values('RECOVERED')
fig = px.bar(df2, x='EPIDEMIC', y='RECOVERED',color = "EPIDEMIC")
recovered_bar=fig

df2 = df.sort_values('RECOVERED')
fig = px.bar(df2, x='EPIDEMIC', y='RECOVERED',color = "EPIDEMIC",log_y=True)
recovered_log_bar=fig

df2 = df.sort_values('FATILITY RATE')
fig = px.bar(df2, x='EPIDEMIC', y='FATILITY RATE',color = "EPIDEMIC")
fatality_bar=fig

df2 = df.sort_values('NUMBER OF COUNTRIES',ascending=False)
fig = px.bar(df2, x='NUMBER OF COUNTRIES', y='EPIDEMIC',color = "EPIDEMIC", orientation='h')
countries_bar=fig

#############################################################################################################################
#############################################################################################################################
############################                                 SCATTERPLOTS                   #################################
#############################################################################################################################

fig = px.scatter(df, x="CONFIRMED", y="DEATHS",size="FATILITY RATE", color="EPIDEMIC", hover_name="EPIDEMIC", log_x=True, log_y=True)
confirmedvsdeath_scatter=fig

fig = px.scatter(df, x="CONFIRMED", y="RECOVERED",size="FATILITY RATE", color="EPIDEMIC", hover_name="EPIDEMIC", log_x=True, log_y=True)
confirmedvsrecovered_scatter=fig

df2 = df.sort_values('NUMBER OF COUNTRIES',ascending=False)
fig = px.scatter(df2, x='NUMBER OF COUNTRIES', y='EPIDEMIC')
epidemicvscountry_scatter=fig

# AREA GRAPHS
df2 = df.sort_values('DEATHS')
fig = px.area(df2, x='EPIDEMIC', y='CONFIRMED',color = "EPIDEMIC")
confirmedvsepidemic_area=fig

df2 = df.sort_values('DEATHS')
fig = px.area(df2, x='EPIDEMIC', y='CONFIRMED',color = "EPIDEMIC",log_y=True)
confirmedvsepidemic_log_area=fig

df2 = df.sort_values('NUMBER OF COUNTRIES',ascending=False)
fig = px.area(df2, x='EPIDEMIC', y='NUMBER OF COUNTRIES')
countries_area=fig

df2 = df.sort_values('FATILITY RATE')
fig = px.area(df2, x='EPIDEMIC', y='FATILITY RATE')
fatality_area=fig

#############################################################################################################################
#############################################################################################################################
############################                            COVID                               #################################
#############################################################################################################################

covid = pd.read_csv(r"C:\Users\vansh\Desktop\DV-CSE3020\DV_Project\DV_Project\owid-covid-data.csv",parse_dates=['date'])
covid = covid[['continent', 'location', 'date','total_cases', 'new_cases','new_cases_smoothed','total_deaths','new_deaths','total_vaccinations']]
# covid.columns = ['Date', 'Country', 'Cases', 'Deaths', 'Recovered']
covid = covid.groupby(['date', 'location'])
covid = covid.sum().reset_index()
covid['total_cases'] = covid['total_cases'].fillna(0)
covid['total_deaths'] = covid['total_deaths'].fillna(0)
covid['total_vaccinations'] = covid['total_vaccinations'].fillna(0)
covid_latest = covid[covid['date'] == max(covid['date'])].reset_index()
covid_latest=covid_latest.sort_values(by=['total_cases'],ascending=False)

temp = covid[['date', 'total_cases', 'total_deaths']]
temp = temp.melt(id_vars='date', value_vars=[ 'total_cases','total_deaths'], var_name='Variable', value_name='Count')
case_death_line=px.line(temp, x='date', y='Count', color='Variable',log_y=True)

fig = px.scatter(covid_latest, x='total_cases', y='total_deaths', color='total_cases',
height=700, hover_name='location',
title='Confirmed vs Deaths',
color_discrete_sequence=px.colors.qualitative.Vivid)
fig.update_traces(textposition='top center')
case_country_scatter=fig

fig = px.scatter(covid_latest, x='total_cases', y='total_deaths', color='total_cases',
height=700, hover_name='location', log_x=True, log_y=True,
title='Confirmed vs Deaths',
color_discrete_sequence=px.colors.qualitative.Vivid)
fig.update_traces(textposition='top center')
case_country_log_scatter=fig

def plot_treemap(col):
    fig = px.treemap(covid_latest, path=["location"], values=col, height=700,
                 title=col, color_discrete_sequence = px.colors.qualitative.Dark2)
    fig.data[0].textinfo = 'label+text+value'
    return fig

covid_cases=plot_treemap('total_cases')
covid_deaths=plot_treemap('total_deaths')

fig = px.scatter(covid_latest.sort_values('total_deaths', ascending=False).iloc[:20, :], 
                 x='total_cases', y='total_deaths', color='location', size='total_cases', 
                 height=700, text='location', log_x=True, log_y=True, 
                 title='Deaths vs Confirmed (Scale is in log10)')
fig.update_traces(textposition='top center')
fig.update_layout(showlegend=False)
fig.update_layout(xaxis_rangeslider_visible=True)
deathconfirmed=fig

fig = px.scatter(covid_latest, x='total_cases', y='total_deaths', color='total_cases',
height=700, hover_name='location', log_x=True, log_y=True,
title='Confirmed vs Deaths',
color_discrete_sequence=px.colors.qualitative.Vivid)
fig.update_traces(textposition='top center')
newscatter=fig

C:\Users\vansh\AppData\Local\Temp/ipykernel_19836/4136819422.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\vansh\AppData\Local\Temp/ipykernel_19836/4136819422.py:7: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
import dash
import dash_bootstrap_components as dbc
from dash import Input, Output, dcc, html
import base64

a = dash.Dash()
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.FONT_AWESOME, dbc.icons.BOOTSTRAP])
app1 = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app2 = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app3 = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app4 = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app5 = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app6 = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app10 = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

image_filename = 'COVID.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

a.layout = html.Div([
    html.H1(children='Coronavirus (COVID-19) Dashboard'),
    
    html.Div(children='''
        Globally, as of 7:55pm CET, 7 December 2021, there have been 265,713,467 confirmed cases of COVID-19,
including 5,260,888 deaths, reported to WHO. As of 5 December 2021, a total of 7,952,750,402 vaccine
doses have been administered.
Welcome to the website. Millions of individuals are being affected by the fast spread of the COVID-19 virus throughout the world.
Our goal is to compile reliable COVID-19-related information that is simple to find, comprehend, and act on.
Here, we'll try to explain how to explore enormous volumes of data, and we'll keep this website updated when new evidence becomes available.
The individuals we've been working with are our target audiences: journalists, non-governmental organisations,
and residents of the communities we're working with. Others may find this valuable as well.
This content will be expanded, modified, and updated on a regular basis. Please come back as frequently as you can and let us know how we can make this site more useful.
    '''),
    #html.Img(src='data:image/png;base64,{}'.format(encoded_image))
    html.Img(src=app.get_asset_url('COVID3.png'))
])

app2.layout = html.Div([
    dcc.Tabs(id='tabs-example', value='tab-1', children=[
        dcc.Tab(label='COVID - Total Cases', value='tab-1'),
        dcc.Tab(label='Other Pandemics - Total Cases', value='tab-2'),
    ]),
    html.Div(id='tabs-example-content-1')
])

@app.callback(
    Output('tabs-example-content-1', 'children'),
    Input('tabs-example', 'value')
)
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.H5('Covid 19-is a contagious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The first known case was identified in Wuhan, China, in December 2019.'),
            dcc.Graph(
                figure=covid_treemap
            )
        ])
    elif tab == 'tab-2':
        return html.Div(children=[
    html.Div([
        html.H3(children='H1N1'),

        html.Div(children='''
            H1N1 flu (swine flu)
            In the spring of 2009, scientists recognized a particular strain of flu virus known as H1N1.
            It is primarily caused by the H1N1 strain of the flu (influenza) virus.
        '''),

        dcc.Graph(
            id='graph1',
            figure=h1n1_treemap
        ),  
    ]),
    # New Div for all elements in the new 'row' of the page
    html.Div([
        html.H3(children='MERS'),

        html.Div(children='''
        A viral respiratory illness caused by a coronavirus.
        It is a contagious, sometimes fatal respiratory illness. It's often spread through close contact with an infected person.
        '''),

        dcc.Graph(
            id='graph2',
            figure=mers_treemap
        ),  
    ]),
     # New Div for all elements in the new 'row' of the page
    html.Div([
        html.H3(children='EBOLA'),

        html.Div(children='''
        A virus that causes severe bleeding, organ failure and can lead to death.
        Humans may spread the virus to other humans through contact with bodily fluids such as blood.
        '''),

        dcc.Graph(
            id='graph3',
            figure=ebola_treemap
        ),  
    ]),
     # New Div for all elements in the new 'row' of the page
    html.Div([
        html.H3(children='SARS'),

        html.Div(children='''
        SARS appeared in 2002 in China. 
        It is a virus transmitted through droplets that enter the air when someone with the disease coughs, sneezes or talks.
        '''),

        dcc.Graph(
            id='graph4',
            figure=sars_treemap
        ),  
    ]),
])
    
app1.layout = html.Div([
    dcc.Tabs(id='tabs-example1', value='tab-1', children=[
        dcc.Tab(label='COVID', value='tab-1'),
        dcc.Tab(label='Other Pandemics', value='tab-2'),
    ]),
    html.Div(id='tabs-example-content-2')
])

@app.callback(
    Output('tabs-example-content-2', 'children'),
    Input('tabs-example1', 'value')
)
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.H5('Covid 19-is a contagious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The first known case was identified in Wuhan, China, in December 2019.'),
            dcc.Graph(
                figure=covid_geospatialmap
            )
        ])
    elif tab == 'tab-2':
        return html.Div(children=[
    html.Div([
        html.H3(children='H1N1'),

        html.Div(children='''
            H1N1 flu (swine flu)
            In the spring of 2009, scientists recognized a particular strain of flu virus known as H1N1.
            It is primarily caused by the H1N1 strain of the flu (influenza) virus.
        '''),

        dcc.Graph(
            id='graph11',
            figure=h1n1_geospatialmap
        ),  
    ]),
    # New Div for all elements in the new 'row' of the page
    html.Div([
        html.H3(children='MERS'),

        html.Div(children='''
        A viral respiratory illness caused by a coronavirus.
        It is a contagious, sometimes fatal respiratory illness. It's often spread through close contact with an infected person.
        '''),

        dcc.Graph(
            id='graph21',
            figure=mers_geospatialmap
        ),  
    ]),
     # New Div for all elements in the new 'row' of the page
    html.Div([
        html.H3(children='EBOLA'),

        html.Div(children='''
        A virus that causes severe bleeding, organ failure and can lead to death.
        Humans may spread the virus to other humans through contact with bodily fluids such as blood.
        '''),

        dcc.Graph(
            id='graph31',
            figure=ebola_geospatialmap
        ),  
    ]),
     # New Div for all elements in the new 'row' of the page
    html.Div([
        html.H3(children='SARS'),

        html.Div(children='''
        SARS appeared in 2002 in China. 
        It is a virus transmitted through droplets that enter the air when someone with the disease coughs, sneezes or talks.
        '''),

        dcc.Graph(
            id='graph41',
            figure=sars_geospatialmap
        ),  
    ]),
])

app3.layout = html.Div([
    dcc.Tabs(id='tabs-example3', value='tab-5', children=[
        dcc.Tab(label='Confirmed Cases', value='tab-1'),
        dcc.Tab(label='Recovered Cases', value='tab-2'),
        dcc.Tab(label='Death Cases', value='tab-3'),
        dcc.Tab(label='Fatality Cases', value='tab-4'),
        dcc.Tab(label='Number of Countries', value='tab-5'),
    ]),
    html.Div(id='tabs-example-content-3')
])

@app.callback(
    Output('tabs-example-content-3', 'children'),
    Input('tabs-example3', 'value')
)
def render_content(tab):
    if tab == 'tab-1':
        return html.Div(children=[
    html.Div([
        html.H3(children='Actual Graph:'),
        dcc.Graph(
            id='graph112',
            figure=confirmed_bar
        ),  
    ]),
    # New Div for all elements in the new 'row' of the page
    html.Div([
        html.H3(children='Logarithmic Graph:'),
        dcc.Graph(
            id='graph222',
            figure=confirmed_log_bar
        ),  
    ]),
])
    elif tab == 'tab-2':
        return html.Div(children=[
    html.Div([
        html.H3(children='Actual Graph:'),
        dcc.Graph(
            id='graph113',
            figure=recovered_bar
        ),  
    ]),
    # New Div for all elements in the new 'row' of the page
    html.Div([
        html.H3(children='Logarithmic Graph:'),
        dcc.Graph(
            id='graph224',
            figure=recovered_log_bar
        ),  
    ]),
])
    elif tab == 'tab-3':
        return html.Div(children=[
    html.Div([
        html.H3(children='Actual Graph:'),
        dcc.Graph(
            id='graph113',
            figure=death_bar
        ),  
    ]),
    # New Div for all elements in the new 'row' of the page
    html.Div([
        html.H3(children='Logarithmic Graph:'),
        dcc.Graph(
            id='graph224',
            figure=death_log_bar
        ),  
    ]),
])
    elif tab == 'tab-4':
        return html.Div(children=[
    html.Div([
        #html.H3(children='Actual Graph:'),
        dcc.Graph(
            id='graph113',
            figure=fatality_bar
        ),  
    ]),
])
    elif tab == 'tab-5':
        return html.Div(children=[
    html.Div([
        #html.H3(children='Actual Graph:'),
        dcc.Graph(
            id='graph113',
            figure=countries_bar
        ),  
    ]),
])
    
app4.layout = html.Div([
    dcc.Tabs(id='tabs-example4', value='tab-1', children=[
        dcc.Tab(label='Confirmed vs Death', value='tab-1'),
        dcc.Tab(label='Confirmed vs Recovered', value='tab-2'),
        dcc.Tab(label='Epidemic vs Country', value='tab-3'),
    ]),
    html.Div(id='tabs-example-content-4')
])

@app.callback(
    Output('tabs-example-content-4', 'children'),
    Input('tabs-example4', 'value')
)
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            #html.H3('Confirmed vs Death'),
            dcc.Graph(
                figure=confirmedvsdeath_scatter
            )
        ])
    elif tab == 'tab-2':
        return html.Div(children=[
    html.Div([
        #html.H3(children='Confirmed vs Recovered'),
        dcc.Graph(
            id='graph122112',
            figure=confirmedvsrecovered_scatter
        ),  
    ]),
])
    elif tab == 'tab-3':
        return html.Div(children=[
    html.Div([
        #html.H3(children='Epidemic vs Country'),
        dcc.Graph(
            id='graph12121',
            figure=epidemicvscountry_scatter
        ),  
    ]),
])
app5.layout = html.Div([
    dcc.Tabs(id='tabs-example5', value='tab-1', children=[
        dcc.Tab(label='Confirmed vs Epidemic', value='tab-1'),
        dcc.Tab(label='Countries', value='tab-2'),
        dcc.Tab(label='Fatality', value='tab-3'),
    ]),
    html.Div(id='tabs-example-content-5')
])

@app.callback(
    Output('tabs-example-content-5', 'children'),
    Input('tabs-example5', 'value')
)
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            dcc.Graph(
                figure=confirmedvsepidemic_log_area
            )
        ])
    elif tab == 'tab-2':
        return html.Div(children=[
    html.Div([
        dcc.Graph(
            id='graph122112123',
            figure=countries_area
        ),  
    ]),
])
    elif tab == 'tab-3':
        return html.Div(children=[
    html.Div([
        dcc.Graph(
            id='graph12121123',
            figure=fatality_area
        ),  
    ]),
])
app10.layout = html.Div([
    dcc.Tabs(id='tabs-example10', value='tab-1', children=[
        dcc.Tab(label='Regression', value='tab-1'),
        dcc.Tab(label='Tree Maps', value='tab-2'),
        dcc.Tab(label='Scatterplot', value='tab-3'),
        
    ]),
    html.Div(id='tabs-example-content-10')
])

@app.callback(
    Output('tabs-example-content-10', 'children'),
    Input('tabs-example10', 'value')
)
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.H4('Regression - New Cases vs New Deaths'),
            html.Img(src=app.get_asset_url('reg1.jpeg')),
            html.H4('Regression - Total Cases vs Total Deaths'),
            html.Img(src=app.get_asset_url('reg2.jpeg'))
        ])
    elif tab == 'tab-2':
        return html.Div(children=[
    html.Div([
        html.H3(children='Covid Deaths'),
        dcc.Graph(
            id='graph1111111',
            figure=covid_deaths
        ),  
    ]),
     # New Div for all elements in the new 'row' of the page
    html.Div([
        html.H3(children='Covid Cases'),
        dcc.Graph(
            id='graph4',
            figure=covid_cases
        ),  
    ]),
])
    elif tab == 'tab-3':
        return html.Div(children=[
    html.Div([
        html.H3(children='Deaths VS Confirmed Cases'),
        dcc.Graph(
            id='graph1111111',
            figure=deathconfirmed
        ),  
    ]),
     # New Div for all elements in the new 'row' of the page
    html.Div([
        dcc.Graph(
            id='graph44444444',
            figure=newscatter
        ),  
    ])
])

# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

sidebar = html.Div(
    [
        html.H2("COVID DASH", className="display-4"),
        html.Hr(),
        html.P(
            "Abhinav Jaiswal - 20BCE2624", className="lead"
        ),
        html.P(
            "Vanshit Kandoi - 20BCE2667", className="lead"
        ),
        html.P(
            "Sneha Jayshri - 20BCE2673", className="lead"
        ),
        dbc.Nav(
            [
                dbc.NavLink("Home", href="/", active="exact"),
                dbc.NavLink("Comparison - Treemaps", href="/page-1", active="exact"),
                #dbc.NavLink("Comparison - Geospatial Maps", href="/page-2", active="exact"),
                dbc.NavLink("Comparison - Bar Graphs", href="/page-3", active="exact"),
                dbc.NavLink("Comparison - Scatterplots", href="/page-4", active="exact"),
                dbc.NavLink("Comparison - Area Graphs", href="/page-5", active="exact"),
                dbc.NavLink("COVID", href="/page-6", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", children = [], style=CONTENT_STYLE)
app.layout = html.Div([dcc.Location(id="url"), sidebar, content])
@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/":
        return a.layout#html.P("This is the content of the home page!")
    elif pathname == "/page-1":
        return app2.layout
    elif pathname == "/page-2":
        return app1.layout
    elif pathname == "/page-3":
        return app3.layout
    elif pathname == "/page-4":
        return app4.layout
    elif pathname == "/page-5":
        return app5.layout#html.P("Oh cool, this is page 5!")
    elif pathname == "/page-6":
        return app10.layout
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )

if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Dec/2021 12:02:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 12:02:56] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 12:02:56] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 12:02:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 12:04:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 12:04:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 12:04:47] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 12:04:47] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 12:05:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 12:07:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 12:07:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09

In [12]:
# """
# This app creates a simple sidebar layout using inline style arguments and the
# dbc.Nav component.

# dcc.Location is used to track the current location, and a callback uses the
# current location to render the appropriate page content. The active prop of
# each NavLink is set automatically according to the current pathname. To use
# this feature you must install dash-bootstrap-components >= 0.11.0.

# For more details on building multi-page Dash applications, check out the Dash
# documentation: https://dash.plot.ly/urls
# """
# import dash
# import dash_bootstrap_components as dbc
# from dash import Input, Output, dcc, html

# app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.FONT_AWESOME, dbc.icons.BOOTSTRAP])
# app1 = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
# app2 = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
# app3 = dash.Dash(__name__)



# app2.layout = html.Div([
#     dcc.Tabs(id='tabs-example', value='tab-1', children=[
#         dcc.Tab(label='Tab one', value='tab-1'),
#         dcc.Tab(label='Tab two', value='tab-2'),
#     ]),
#     html.Div(id='tabs-example-content-1')
# ])

# @app.callback(
#     Output('tabs-example-content-1', 'children'),
#     Input('tabs-example', 'value')
# )
# def render_content(tab):
#     if tab == 'tab-1':
#         return html.Div([
#             html.H3('Tab content 1'),
#             dcc.Graph(
#                 figure=covid_treemap
# #                 dict(
# #                     data=[dict(
# #                         x=[1, 2, 3],
# #                         y=[3, 1, 2],
# #                         type='bar'
# #                     )]
# #                 )
#             )
#         ])
#     elif tab == 'tab-2':
#         return html.Div([
#             html.H3('Tab content 2'),
#             dcc.Graph(
#                 figure=covid_treemap
# #                 dict(
# #                     data=[dict(
# #                         x=[1, 2, 3],
# #                         y=[5, 10, 6],
# #                         type='bar'
# #                     )]
# #                 )
#             )
#         ])
# # app2.layout = dcc.RadioItems(id = 'radio_items',
# #     options=[
# #         {'label': 'Covid             .', 'value': 'COVID'},
# #         {'label': 'Other Pandemics', 'value': 'PANDEMIC'}
# #     ],
# #     value='COVID',
# #     labelStyle={'display': 'inline-block'}
# # )
# # @app.callback(Output('children', 'figure'), [Input('radio_items', 'value')])
# # def get_pandemic_options(value):
# #     if value=='COVID':
# #         return app1.layout
# #     elif value == "PANDEMIC":
# #         return fig

# app1.layout = html.Div(children=[
#     html.H1(children='Tree Maps'),
# #     html.Div([
# #         [dcc.Location(id="url"), sidebar, content]
# #     ]),
#     # All elements from the top of the page
#     html.Div([
#         html.H3(children='Hello Dash'),

#         html.Div(children='''
#             Dash: A web application framework for Python.
#         '''),

#         dcc.Graph(
#             id='graph1',
#             figure=covid_treemap
#         ),  
#     ]),
#     # New Div for all elements in the new 'row' of the page
#     html.Div([
#         html.H3(children='Hello Dash'),

#         html.Div(children='''
#             Dash: A web application framework for Python.
#         '''),

#         dcc.Graph(
#             id='graph2',
#             figure=covid_treemap
#         ),  
#     ]),
#      # New Div for all elements in the new 'row' of the page
#     html.Div([
#         html.H3(children='Hello Dash'),

#         html.Div(children='''
#             Dash: A web application framework for Python.
#         '''),

#         dcc.Graph(
#             id='graph3',
#             figure=covid_treemap
#         ),  
#     ]),
#      # New Div for all elements in the new 'row' of the page
#     html.Div([
#         html.H3(children='Hello Dash'),

#         html.Div(children='''
#             Dash: A web application framework for Python.
#         '''),

#         dcc.Graph(
#             id='graph4',
#             figure=covid_treemap
#         ),  
#     ]),
# ])
# # the style arguments for the sidebar. We use position:fixed and a fixed width
# SIDEBAR_STYLE = {
#     "position": "fixed",
#     "top": 0,
#     "left": 0,
#     "bottom": 0,
#     "width": "16rem",
#     "padding": "2rem 1rem",
#     "background-color": "#f8f9fa",
# }

# # the styles for the main content position it to the right of the sidebar and
# # add some padding.
# CONTENT_STYLE = {
#     "margin-left": "18rem",
#     "margin-right": "2rem",
#     "padding": "2rem 1rem",
# }

# sidebar = html.Div(
#     [
#         html.H2("COVID-19", className="display-4"),
#         html.Hr(),
#         html.P(
#             "A simple sidebar layout with navigation links", className="lead"
#         ),
#         dbc.Nav(
#             [
#                 dbc.NavLink("Home", href="/", active="exact"),
#                 dbc.NavLink("Comparison", href="/page-1", active="exact"),
#                 dbc.NavLink("Page 2", href="/page-2", active="exact"),
#                 dbc.NavLink("Page 3", href="/page-3", active="exact"),
#                 dbc.NavLink("Page 4", href="/page-4", active="exact"),
#                 dbc.NavLink("Page 5", href="/page-5", active="exact"),
#             ],
#             vertical=True,
#             pills=True,
#         ),
#     ],
#     style=SIDEBAR_STYLE,
# )

# content = html.Div(id="page-content", children = [], style=CONTENT_STYLE)

# app.layout = html.Div([dcc.Location(id="url"), sidebar, content])


# @app.callback(Output("page-content", "children"), [Input("url", "pathname")])
# def render_page_content(pathname):
#     if pathname == "/":
#         return html.P("This is the content of the home page!")
#     elif pathname == "/page-1":
#         return app2.layout#html.P("This is the content of page 1. Yay!")
#     elif pathname == "/page-2":
#         return html.P("Oh cool, this is page 2!")
#     elif pathname == "/page-3":
#         return html.P("Oh cool, this is page 3!")
#     elif pathname == "/page-4":
#         return html.P("Oh cool, this is page 4!")
#     elif pathname == "/page-5":
#         return html.P("Oh cool, this is page 5!")
#     # If the user tries to reach a different page, return a 404 message
#     return dbc.Jumbotron(
#         [
#             html.H1("404: Not found", className="text-danger"),
#             html.Hr(),
#             html.P(f"The pathname {pathname} was not recognised..."),
#         ]
#     )


# if __name__ == "__main__":
#     app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Dec/2021 01:13:45] "GET /page-3 HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 01:13:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 01:13:45] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 01:13:45] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 01:13:45] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\vansh\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\vansh\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\vansh\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\vansh\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\vansh\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\vansh\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\vansh\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\vansh\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

127.0.0.1 - - [09/Dec/2021 01:14:37] "POST /_dash-update-component HTTP/1.1" 500 -


In [6]:

# import dash
# import dash_bootstrap_components as dbc
# from dash import Input, Output, dcc, html

# app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
# app.layout = html.Div(children=[
#     # All elements from the top of the page
#     html.Div([
#         html.H1(children='Hello Dash'),

#         html.Div(children='''
#             Dash: A web application framework for Python.
#         '''),

#         dcc.Graph(
#             id='graph1',
#             figure=confirmedvsepidemic_log_area
#         ),  
#     ]),
# ])

# if __name__ == '__main__':
#     app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Dec/2021 00:55:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 00:55:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 00:55:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 00:55:51] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2021 00:55:51] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
